# Plant Health AI Assistant
## 概要
このノートブックでは、AWS Bedrock AgentCore を使用して、永続的なメモリ機能を備えた LangGraph ベースの植物健康解析システムをデプロイする方法を実演します。このシステムはマルチエージェントオーケストレーションとメモリストレージを組み合わせて、インテリジェントな植物診断と履歴追跡を提供します。

## Plant Health AI Assistant - Part 1
この最初のノートブックでは、Bedrock AgentCore Gateway のツールとして使用する5つの Lambda 関数を作成します

# 01. 依存関係のインストールとクライアントの設定

In [ ]:
!pip install -q -r requirements.txt --no-cache-dir

In [ ]:
import boto3
import random
import string
import json
import time
import os
import requests

from utils.utils import create_lambda_role, create_lambda_layer, create_lambda

# コアクライアントと変数
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
region_name = boto3.session.Session().region_name
suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))

TAVILY_API_KEY='<YOUR API KEY>'

# クライアントの初期化
lambda_client = boto3.client('lambda', region_name=region_name)
iam_client = boto3.client('iam', region_name=region_name)
cognito_client = boto3.client('cognito-idp')
bedrock_agentcore_client = boto3.client('bedrock-agentcore-control',
                                       region_name=region_name,
                                       endpoint_url=f"https://bedrock-agentcore-control.{region_name}.amazonaws.com")

print(f"アカウント: {account_id}, リージョン: {region_name}, サフィックス: {suffix}")

# 02. Lambda 関数の作成

In [ ]:
lambda_role_name = 'PlantAdvisorLambdaRole'

role_response = create_lambda_role(lambda_role_name)

lambda_role_arn = role_response['Role']['Arn']
print(f"LambdaロールARN: {lambda_role_arn}")


In [ ]:
# 汎用作成関数を使用して5つのLambda関数をすべてデプロイ
lambda_code_path = "./Lambda"
lambda_list = []

for lambda_code in os.listdir(lambda_code_path):
    if os.path.isfile(os.path.join(lambda_code_path, lambda_code)):
        lambda_name = os.path.splitext(lambda_code)[0]
        with open(os.path.join(lambda_code_path, lambda_code), 'r') as f:
            globals()[lambda_name] = f.read()
            lambda_list.append(lambda_name)


# 作成されたすべての変数とその内容を表示
for lambda_name in lambda_list:
    
    lambda_code=globals()[lambda_name]

    if lambda_name=='plant_detection':
        create_lambda(lambda_name, lambda_code, lambda_role_arn, 'Plant detection')
    
    if lambda_name=='plant_care':
        create_lambda(lambda_name, lambda_code, lambda_role_arn, 'Plant care advice')

    if lambda_name=='plant_websearch':
        create_lambda(lambda_name, lambda_code, lambda_role_arn, 'Plant web search', TAVILY_API_KEY)

    if lambda_name=='websearch':
        packages=["requests"]
        create_lambda(lambda_name, lambda_code, lambda_role_arn, 'Websearch', TAVILY_API_KEY, packages=packages)
    
    if lambda_name=='weather_forecast':
        packages=["requests"]
        create_lambda(lambda_name, lambda_code, lambda_role_arn, 'Get Weather Forecast', packages=packages)

    print(f"✅ {lambda_name} Lambda関数を作成しました")

print(f"✅ {len(lambda_list)}個のLambda関数を作成しました")

# 03. Starter Toolkit を使用した Bedrock AgentCore Gateway の作成

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
import logging

# クライアントのセットアップ
client = GatewayClient(region_name=region_name)
client.logger.setLevel(logging.DEBUG)

# Cognito オーソライザーの作成
cognito_response = client.create_oauth_authorizer_with_cognito("PlantGatewayAuth")

# ゲートウェイの作成
gateway = client.create_mcp_gateway(authorizer_config=cognito_response["authorizer_config"])

# ゲートウェイオブジェクトからゲートウェイIDを取得
gateway_id = gateway.get('gatewayId') or gateway.get('gatewayArn').split('/')[-1]
print(f"ゲートウェイID: {gateway_id}")

# ゲートウェイURLを取得
gateway_url = gateway.get('gatewayUrl')
print(f"ゲートウェイURL: {gateway_url}")


# アクセストークンを取得
access_token = client.get_access_token_for_cognito(cognito_response["client_info"])

# 04. Bedrock AgentCore Gateway に MCP ツールとして Lambda 関数を登録

In [ ]:
def create_target(name, lambda_arn, tool_name, tool_desc, properties):
    """汎用ターゲット作成関数"""
    try:
        config = {
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {
                        "inlinePayload": [{
                            "name": tool_name,
                            "description": tool_desc,
                            "inputSchema": {
                                "type": "object",
                                "properties": properties
                            }
                        }]
                    }
                }
            }
        }
        
        response = bedrock_agentcore_client.create_gateway_target(
            gatewayIdentifier=gateway_id,
            name=name,
            targetConfiguration=config,
            credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
        )
        
        print(f"✅ ターゲット {name} を作成しました")
        return response['targetId']
    
    except Exception as e:
        print(f"❌ ターゲット作成中にエラー: {e}")
        return None

# すべてのターゲットを作成

for lambda_name in lambda_list:
    time.sleep(10)

    response = lambda_client.get_function(FunctionName=lambda_name)
    lambda_arn = response['Configuration']['FunctionArn']

    if lambda_name == 'plant_detection':
        create_target('plant-detection-target', 
                      lambda_arn, 
                      'plant_detection_tool', 
                      'Detect plant and health', 
                      {"image_data": {"type": "string"}})
        
    if lambda_name == 'plant_care':
        create_target('plant-care-target', 
                      lambda_arn,
                      'plant_care_tool', 
                      'Get plant care advice',
                      {"plant_name": {"type": "string"}, "health_status": {"type": "string"}})
    
    if lambda_name == 'plant_websearch':
        create_target('plant-search-target', 
                      lambda_arn,
                      'plant_web_search_tool', 
                      'Search plant information',
                      {"plant_name": {"type": "string"}, "health_status": {"type": "string"}})

    if lambda_name == 'websearch':
        create_target('websearch-target', 
                      lambda_arn,
                      'websearch_tool', 
                      'Generic web search',
                      {"query": {"type": "string"}})

    if lambda_name == 'weather_forecast':
        create_target('weather-forecast-target', 
                      lambda_arn,
                      'weather-forecast-tool', 
                      'Get weather forecast for a specific location',
                      {"location": {"type": "string"}})


time.sleep(15)  # ターゲットの準備完了を待機
print(f"✅ {len(lambda_list)}個のゲートウェイターゲットを作成しました")

## MCP ツールのセットアップとテスト

In [ ]:
# Gateway 情報を取得し MCP ツールをテストするためのユーティリティをセットアップ

def get_gateway_info():
    """Gateway URL と利用可能なツールを取得"""
    gateway_info = bedrock_agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = gateway_info.get('gatewayUrl')
    
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {access_token}'}
    response = requests.post(gateway_url, headers=headers, 
                           json={"jsonrpc": "2.0", "id": 1, "method": "tools/list", "params": {}})
    
    tools = {}
    if response.status_code == 200:
        result = response.json()
        if 'result' in result and 'tools' in result['result']:
            for tool in result['result']['tools']:
                name = tool.get('name', '')
                if 'plant-detection' in name:
                    tools['detection'] = name
                elif 'plant-care' in name:
                    tools['care'] = name
                elif 'plant-search' in name:
                    tools['search'] = name
                elif 'weather-forecast' in name:
                    tools['weather'] = name
    
    return gateway_url, tools

def test_tool(gateway_url, tool_name, arguments):
    """汎用ツールテスト関数"""
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {access_token}'}
    payload = {
        "jsonrpc": "2.0", "id": 2, "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments}
    }
    
    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json() if response.status_code == 200 else None

In [ ]:
# ローカル画像ファイルを読み込んでエンコード
import base64
from pprint import pprint
gateway_url, available_tools = get_gateway_info()
print(f"✅ ゲートウェイURL: {gateway_url}")
print(f"✅ 利用可能なツール: {list(available_tools.keys())}")

with open('./Image/sweet_potato_leaf.png', 'rb') as f:
    image_data = base64.b64encode(f.read()).decode('utf-8')
    print(f"✅ 画像をエンコードしました: {len(image_data)}文字")

# すべてのツールをテスト
test_cases = [
    ('detection', {'image_data': image_data}),
    ('care', {'plant_name': 'sweet potato leaf', 'health_status': 'yellowing leaves'}),
    ('search', {'plant_name': 'sweet potato leaf', 'health_status': 'pest damage'})
]

for test_type, args in test_cases:
    if test_type in available_tools:
        print(f"\n🧪 {test_type} をテスト中...")
        result = test_tool(gateway_url, available_tools[test_type], args)
        if result:
            print(f"✅ {test_type.title()} テストが成功しました！")
            pprint(result)
        else:
            print(f"❌ {test_type.title()} テストが失敗しました")
        time.sleep(10)

print(f"\n🎉 植物解析ゲートウェイのセットアップが完了しました！")
print(f"ゲートウェイID: {gateway_id}")
print(f"ゲートウェイURL: {gateway_url}")

# 05. 再利用のための設定を保存

In [ ]:

config_data = {
    "COGNITO_AUTH_CONFIG": cognito_response["authorizer_config"],
    "COGNITO_INFO": cognito_response["client_info"],
    "REGION": region_name,
    "GATEWAY_ID": gateway_id,
    "GATEWAY_URL": gateway_url,
    "AVAILABLE_TOOLS": available_tools
}

# JSON ファイルに保存
with open('plant_gateway_config.json', 'w') as f:
    json.dump(config_data, f, indent=2)

print("✅ 設定を 'plant_gateway_config.json' に保存しました")
print("📋 LangGraph ノートブックでこれを使用してください！")